<a href="https://colab.research.google.com/github/muneesh-07/MachineLearning_project/blob/main/BertTextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install transformers datasets torch

In [18]:
from transformers import BertTokenizer, BertForSequenceClassification


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
text = "I Love this Product!"

input = tokenizer(text,return_tensors="pt",padding=True,truncation = True,max_length = 128)

In [20]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

dataset = load_dataset("imdb")
train_data = dataset["train"].shuffle(seed=42).select(range(2000))
test_data = dataset["test"].shuffle(seed=42).select(range(500))

def tokenize_function(examples):
  return tokenizer(examples['text'],padding='max_length',truncation=True)

train_data = train_data.map(tokenize_function,batched=True)
test_data = test_data.map(tokenize_function,batched=True)


training_args = TrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    num_train_epochs = 3,
    weight_decay = 0.01,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_data,
    eval_dataset = test_data,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.269692
2,No log,0.300287
3,No log,0.323819


TrainOutput(global_step=375, training_loss=0.2656890869140625, metrics={'train_runtime': 626.4653, 'train_samples_per_second': 9.578, 'train_steps_per_second': 0.599, 'total_flos': 1578666332160000.0, 'train_loss': 0.2656890869140625, 'epoch': 3.0})

In [22]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Move model to the correct device
model = model.to(device)

# Test with a new sentence
test_text = "The movie was fantastic and full of surprises!"

# Tokenize input and move to device
inputs = tokenizer(test_text, return_tensors="pt", padding="max_length", truncation=True).to(device)

# Perform inference
outputs = model(**inputs)

# Get the prediction
prediction = outputs.logits.argmax(-1).item()
print("Prediction:", "Positive" if prediction == 1 else "Negative")


Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prediction: Negative
